We've focused on 1D (Series) and 2D (Dataframes) data, but it would be useful to store higher-dimensional data (indexed by more than one or two keys).
Pands does provide `Panel` and `Panel4D` object to store 3D and 4D data, a far more common pattern is to make use of hierarchical indexing (multi-indexing).

In [1]:
import pandas as pd
import numpy as np

# A Multiple Indexed Series
## The bad way

In [2]:
index = [('California', 2000), ('California', 2010), ('New York', 2000),
        ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

## The better way: Pandas MultiIndex

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [10]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


# Methods of MultiIndex Creation


In [12]:
# create a data frame with multiple indexes
df = pd.DataFrame(np.random.rand(4, 2),
                 index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                 columns=['data1', 'data2'])
df

data1     data2
a 1  0.668898  0.764226
  2  0.410655  0.188449
b 1  0.051716  0.700024
  2  0.576307  0.486232

In [13]:
# pass a dictionary with appropriate tuples as keys, Pandas will recognize this and use MultiIndex by default
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

## Explicit MultiIndex constructors

In [14]:
# From arrays
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1,2,1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [15]:
# From Tuples
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [16]:
# From cartesian product
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [18]:
# levels and labels
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

## MultiIndex level names

In [20]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## MultiIndex for Columns

In [21]:
# hierarchical indices and columns
# 4D data: subject, types, year and visit
index = pd.MultiIndex.from_product([[2013, 2014], [1,2]],
                                  names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                    names=['subject', 'types'])

# mock some data
data = np.round(np.random.randn(4,6), 1)
data[:,::2] *= 10
data += 37

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
types         HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      25.0  37.7  42.0  38.4  28.0  38.4
     2      44.0  36.1  36.0  36.3  28.0  37.7
2014 1      29.0  36.5  18.0  37.1  47.0  37.7
     2      44.0  34.5  28.0  36.7  22.0  35.2

In [22]:
health_data.Guido

types         HR  Temp
year visit            
2013 1      42.0  38.4
     2      36.0  36.3
2014 1      18.0  37.1
     2      28.0  36.7

# Indexing and Slicing a MultiIndex
## Multiply indexed Series

In [23]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [27]:
pop['California', 2000]

33871648

In [28]:
pop.loc['California': 'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [29]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [30]:
pop[pop> 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

## Multiply indexed DataFrames

In [31]:
health_data

subject      Bob       Guido         Sue      
types         HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      25.0  37.7  42.0  38.4  28.0  38.4
     2      44.0  36.1  36.0  36.3  28.0  37.7
2014 1      29.0  36.5  18.0  37.1  47.0  37.7
     2      44.0  34.5  28.0  36.7  22.0  35.2

In [32]:
health_data['Guido', 'HR']

year  visit
2013  1        42.0
      2        36.0
2014  1        18.0
      2        28.0
Name: (Guido, HR), dtype: float64

In [33]:
health_data.iloc[:2, :2]

subject      Bob      
types         HR  Temp
year visit            
2013 1      25.0  37.7
     2      44.0  36.1